In [1]:
import numpy as np
import pandas as pd 

In [9]:
class CONFIG:
    SEED = 67
    TRAGET = 'diagnosed_diabetes'

    TRAIN_PATH = '/kaggle/input/playground-series-s5e12/train.csv'
    TEST_PATH = '/kaggle/input/playground-series-s5e12/test.csv'

cfg = CONFIG()

In [41]:
def print_desgin(data):
    print('=' * 25)
    if data:
        print(data)
        print('=' * 25, end='\n')

In [42]:
def import_data(cfg: CONFIG = cfg, sep_ids=True):
    print_desgin("1. IMPORTING DATASET")
    train = pd.read_csv(cfg.TRAIN_PATH)
    test = pd.read_csv(cfg.TEST_PATH)
    print('\n- Successfully imported the datasets\n')

    test_ids = None
    if sep_ids:
        print('--- Separating and Removing Ids ---\n')
        train = train.drop('id', axis=1)
        test_ids = test.id
        test = test.drop('id', axis=1)
        print('- Successfully done!')
        return train, test, test_ids

    print_desgin()
        
    return train, test

In [43]:
train, test, test_ids = import_data()

1. IMPORTING DATASET

- Successfully imported the datasets

--- Separating and Removing Ids ---

- Successfully done!
